In [2]:
import cv2
import numpy as np
import pandas as pd
import os
import shutil
import random
import torch
from PIL import Image

In [11]:
flower_label = {'rose' : 0,             # 장미
                'sunflower': 1,         # 해바라기
                'tulip' : 2,            # 튤립
                'carnation' : 3,        # 카네이션
                'gerbera' : 4,          # 거베라
                'lily' : 5,             # 백합
                'lisianthus' : 6,      # 리시안셔스
                'chrysanthemum' : 7,    # 소국
                'peony' : 8,            # 작약
                'alstroemeria' : 9,     # 알스트로메리아
                'ranunculus' : 10,      # 라넌큘러스
                'freesia' : 11,          # 프리지아
                'iris' : 12,            # 붓꽃 (아이리스)
                'hydrangea' : 13,       # 수국
                'pompom blooms' :14,    # 퐁퐁국화
                'stock' : 15,           # 스톡
                'spary_rose' : 16,      # 스프레이 장미
                'ranunculus_butterfly':17,  # 라넌큘러스 버터플라이
                'delpinium' : 18        # 델피늄
                }

flower = 'lily'

f_num = flower_label[flower]

# 모델 로드

model = torch.hub.load('ultralytics/yolov5', 'custom', path=f'./models/{flower}.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\SSAFY/.cache\torch\hub\master.zip
YOLOv5  2023-4-28 Python-3.11.2 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2070 Super with Max-Q Design, 8192MiB)



requirements: C:\Users\SSAFY\.cache\torch\hub\requirements.txt not found, check failed.


Fusing layers... 
one_YOLOv5m summary: 212 layers, 20873139 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


## 모델로 바운딩 박스 그리기

In [8]:
image_list = os.listdir(f'./0_flower_data/{flower}')

if not os.path.isdir(f'./4_model_images/{flower}'):
    os.mkdir(f'./4_model_images/{flower}')

for f_name in image_list[:]:
    
    try: 
        img = Image.open(f'./0_flower_data/{flower}/{f_name}')
    except:
        os.remove(f'./0_flower_data/{flower}/{f_name}')
        continue
    
    results = model(img)
    
    if len(results.xywh[0]):
        
        for result in results.xywhn[0]:
            x,y,w,h,score,CLS = map(float, result)
        
            with open(f'./1_labels_data/{flower}/{f_name[:-4]}.txt', 'a') as txt:
                txt.write(f'{CLS} {round(x, 4)} {round(y, 4)} {round(w, 4)} {round(h, 4)}\n')
    
    results.save()
    
    shutil.move(f'./runs/detect/exp/{f_name}', f'./4_model_images/{flower}/')
    os.rmdir('./runs/detect/exp')

os.rmdir('./runs/detect')

Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 im

## 데이터 수작업으로 판별해주기

In [9]:
# 마우스 이벤트 콜백 함수
def mouse_callback(event, x, y, flags, param):
    global point1, point2, cnt, img, img2, coordi
    if event == cv2.EVENT_LBUTTONDOWN:
        if cnt%2:
            point1 = (x, y)
            img2 = img.copy()
            cv2.line(img, (x, y), (x, img.shape[0]), (0, 255, 0), 10)
            cv2.line(img, (x, y), (img.shape[1], y), (0, 255, 0), 10)
            cv2.imshow(f'{f_name} {width} * {height}', img)
        else:
            px, py = point1
            cv2.line(img2, (px, py), (px, y), (0, 255, 0), 10)
            cv2.line(img2, (px, py), (x, py), (0, 255, 0), 10)
            cv2.line(img2, (x, y), (px, y), (0, 255, 0), 10)
            cv2.line(img2, (x, y), (x, py), (0, 255, 0), 10)
            cv2.imshow(f'{f_name} {width} * {height}', img2)
            img = img2.copy()
            point2 = (x, y)
            
            if point1 and point2:
                # 좌측상단 좌표와 우측하단 좌표로 변환
                x1, y1 = min(point1[0], point2[0]), min(point1[1], point2[1])
                x2, y2 = max(point1[0], point2[0]), max(point1[1], point2[1])
                
                x = round((x1+(x2-x1)/2)/img.shape[1], 4)
                y = round((y1+(y2-y1)/2)/img.shape[0], 4)
                w = round((x2-x1)/img.shape[1], 4)
                h = round((y2-y1)/img.shape[0], 4)
                
                coordi += f"{f_num} {x} {y} {w} {h}\n"
                # 좌표 저장
        cnt += 1

break_plag = False
flower_list = os.listdir(f'./4_model_images/{flower}')

if not os.path.isdir(f'./1_labels_data/{flower}'):
    os.mkdir(f'./1_labels_data/{flower}')

for f_name in flower_list:
    
    if break_plag:
        break
    
    # 이미지 파일 경로
    img_path = f'./4_model_images/{flower}/{f_name}'

    # 이미지 로드 및 윈도우 생성
    img = cv2.imread(img_path)
    
    if type(img) != np.ndarray:
        os.remove(f'./4_model_images/{flower}/{f_name}')
        continue
    
    height, width = img.shape[:2]
    
    if height > width:
        new_height = 640
        new_width = int(width * (new_height / height))
    else:
        new_width = 640
        new_height = int(height * (new_width / width))
    
    cv2.namedWindow(f'{f_name} {width} * {height}', cv2.WINDOW_NORMAL)
    cv2.resizeWindow(f'{f_name} {width} * {height}', (new_width, new_height))
    cv2.setMouseCallback(f'{f_name} {width} * {height}', mouse_callback)
    cv2.imshow(f'{f_name} {width} * {height}', img)

    # 마우스 이벤트를 통한 좌표 저장
    point1, point2, coordi, cnt = None, None, '', 1
    while True:
        key = cv2.waitKey(1) & 0xFF
        
        # 좌표를 txt 파일로 저장하고 다음으로 넘어가기
        # 만약 좌표를 그리지 않았다면 저장하지 않고 그냥 넘어가기
        if key == ord("q"):
            if coordi == '':
                break
            
            with open(f"./1_labels_data/{flower}/{f_name[:-4]}.txt", "a") as f:
                f.write(coordi)
            break
        
        # 저장하지 않은 좌표들을 지우고, 다시 그리기
        elif key == ord('c'):
            cnt = 1
            img = cv2.imread(img_path)
            coordi = ''
            cv2.imshow(f'{f_name} {width} * {height}', img)
        
        # 볼 것도 없다. 데이터셋에서 그냥 삭제
        elif key == ord('f'):
            os.remove(f'./4_model_images/{flower}/{f_name}')
            break
        
        # 여기서 끝내기!
        elif key == ord('p'):
            break_plag = True
            break

    # 윈도우 종료 및 리소스 해제q
    cv2.destroyAllWindows()

KeyboardInterrupt: 

### (optional) 한 번 더 학습 시키고 싶다!

In [5]:
# 3의 이전 파일들 삭제

for C in ['train', 'test']:
    for L in ['labels', 'images']:
        for i in os.listdir(f'./3_one_time_data/{C}/{L}'):
            os.remove(f'./3_one_time_data/{C}/{L}/{i}')
    
    if os.path.isfile(f'./3_one_time_data/{C}/labels.cache'):
        os.remove(f'./3_one_time_data/{C}/labels.cache')

# 4에 존재하는 이미지의 이름을 비교해, 0과 1의 파일을 3으로 보내기!

bound_img_list = os.listdir(f'./4_model_images/{flower}')

random_list = random.sample([i for i in range(len(bound_img_list))], len(bound_img_list)//10*2)

for i in range(len(bound_img_list)):
    CLS = 'train'
    img = bound_img_list[i]
    txt = f'{img[:-4]}.txt'
    
    if i in random_list:
        CLS = 'test'
        
    if not os.path.isfile(f'./1_labels_data/{flower}/{txt}'):
        os.remove(f'./4_model_images/{flower}/{img}')
        os.remove(f'./0_flower_data/{flower}/{img}')
        continue
    try:
        shutil.copy(f'./0_flower_data/{flower}/{img}', f'./3_one_time_data/{CLS}/images/{img}')
        shutil.copy(f'./1_labels_data/{flower}/{txt}', f'./3_one_time_data/{CLS}/labels/{txt}')
    except:
        print(f'{img}')

## 결측치, 이상치를 제거한 후, 1과 4의 데이터를 5로 옮기자!

In [12]:
import random

bound_img_list = os.listdir(f'./4_model_images/{flower}')

random_list = random.sample([i for i in range(len(bound_img_list))], len(bound_img_list)//10*2)

for i in range(len(bound_img_list)):
    CLS = 'train'
    img = bound_img_list[i]
    txt = f'{img[:-4]}.txt'
    
    if i in random_list:
        CLS = 'test'
        
    if not os.path.isfile(f'./1_labels_data/{flower}/{txt}'):
        os.remove(f'./4_model_images/{flower}/{img}')
        os.remove(f'./0_flower_data/{flower}/{img}')
        continue
    try:
        shutil.move(f'./0_flower_data/{flower}/{img}', f'./5_total_data/{CLS}/images/{img}')
        shutil.move(f'./1_labels_data/{flower}/{txt}', f'./5_total_data/{CLS}/labels/{txt}')
    except:
        print(f'{img}')

python train.py --img 640 --rect --batch 16 --epochs 1 --data total_data.yaml --cfg total_yolov5m.yaml --weight yolov5m.pt --name {flower}_test

### 남은 잔여 파일들 정리

In [7]:
# remain_img = os.listdir(f'./0_flower_data/{flower}')

# for i in range(len(remain_img)):
#     os.remove(f'./0_flower_data/{flower}/{remain_img[i]}')
    
remain_txt = os.listdir(f'./1_labels_data/{flower}')

for i in range(len(remain_txt)):
    os.remove(f'./1_labels_data/{flower}/{remain_txt[i]}')

remain_bound_img = os.listdir(f'./4_model_images/{flower}')

for i in remain_bound_img:
    os.remove(f'./4_model_images/{flower}/{i}')

### 3의 이전 파일들 삭제하기

In [5]:
for C in ['train', 'test']:
    for L in ['labels', 'images']:
        for i in os.listdir(f'./3_one_time_data/{C}/{L}'):
            os.remove(f'./3_one_time_data/{C}/{L}/{i}')
    
    if os.path.isfile(f'./3_one_time_data/{C}/labels.cache'):
        os.remove(f'./3_one_time_data/{C}/labels.cache')

### 5의 데이터세트 중, 한 종류를 3으로 옮기고 싶을 때

In [ ]:
for C in ['train', 'test']:
    for L in ['images', 'labels']:
        f_list = os.listdir(f'./5_total_data/{C}/{L}')
        
        for f in f_list:
            if f[0] == 'r':
                shutil.copy(f'./5_total_data/{C}/{L}/{f}', f'./3_one_time_data/{C}/{L}/{f}')